In [2]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

from data_preprocessing.spatio_temporal_features import SpatioTemporalFeatures
from utils.common import GroupsInfo, get_mapping_cols_tobii, get_mapping_cols, get_mapping_cols_centroids
from config.constants import TRAJECTORY_SAMPLES_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
TRAJECTORY_SAMPLES_PATH = "/home/tmr/Documents/PhD/My_PhD/code/magni-dash/data"

In [9]:
files = os.listdir(os.path.join(TRAJECTORY_SAMPLES_PATH, "Scenario3"))
files_target = list(filter(lambda x: x.endswith(".csv"), files))
files_target

['170522_SC3A_R2.csv', '170522_SC3B_R1.csv']

In [10]:
FILE_IDX = 0

In [11]:
files_target[FILE_IDX]

'170522_SC3A_R2.csv'

In [12]:
sync_df = pd.read_csv(os.path.join(TRAJECTORY_SAMPLES_PATH, "Scenario3", files_target[FILE_IDX]))

In [6]:
sync_df[sync_df.columns[sync_df.columns.str.contains("Helmet_10 Centroid")]].isna().sum() / sync_df.shape[0]

Helmet_10 Centroid_X    0.407083
Helmet_10 Centroid_Y    0.407083
Helmet_10 Centroid_Z    0.407083
dtype: float64

In [7]:
for m in range(1,5):
    print("X", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} X")]].isna().sum() / sync_df.shape[0])
    print("Y", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} Y")]].isna().sum() / sync_df.shape[0])
    print("Z", sync_df[sync_df.columns[sync_df.columns.str.contains(f"Helmet_10 - {m} Z")]].isna().sum() / sync_df.shape[0])

X Helmet_10 - 1 X    0.145767
dtype: float64
Y Helmet_10 - 1 Y    0.145767
dtype: float64
Z Helmet_10 - 1 Z    0.145767
dtype: float64
X Helmet_10 - 2 X    0.204275
dtype: float64
Y Helmet_10 - 2 Y    0.204275
dtype: float64
Z Helmet_10 - 2 Z    0.204275
dtype: float64
X Helmet_10 - 3 X    0.195306
dtype: float64
Y Helmet_10 - 3 Y    0.195306
dtype: float64
Z Helmet_10 - 3 Z    0.195306
dtype: float64
X Helmet_10 - 4 X    0.211945
dtype: float64
Y Helmet_10 - 4 Y    0.211945
dtype: float64
Z Helmet_10 - 4 Z    0.211945
dtype: float64


In [8]:
sync_df[sync_df.columns[sync_df.columns.str.contains("G3D")]].isna().sum() / sync_df.shape[0]

Helmet_10 TB2_G3D_X    0.777242
Helmet_10 TB2_G3D_Y    0.777242
Helmet_10 TB2_G3D_Z    0.777242
Helmet_5 TB3_G3D_X     0.503856
Helmet_5 TB3_G3D_Y     0.503856
Helmet_5 TB3_G3D_Z     0.503856
dtype: float64

In [9]:
def get_best_markers(input_df: pd.DataFrame):
    """Get markers with lowest amount of NaN values"""
    x_coordinate = input_df[input_df.columns[input_df.columns.str.endswith("X")]]
    x_cols = x_coordinate.columns

    instances = set(x_coordinate.columns.str.split(" - ").str[0])
    instances = list(filter(lambda x: len(x.split(" ")) == 1, instances))
    nan_counter_by_marker = {}
    for instance_id in instances:
        nan_counter_by_marker[instance_id] = {}
        markers = (
            x_coordinate[x_cols[x_cols.str.startswith(f"{instance_id} -")]]
            .columns.str.split(regex=r" (/d) ")
            .str[2]
        )
        for marker_id in markers:
            n_nans = (
                x_coordinate[f"{instance_id} - {marker_id} X"]
                .isna()
                .sum()
            )
            nan_counter_by_marker[instance_id][marker_id] = n_nans
    print(nan_counter_by_marker)
    return nan_counter_by_marker

In [10]:
best_markers = get_best_markers(sync_df)

{'Helmet_2': {'1': 3078, '2': 2179, '3': 5822, '4': 5674}, 'Helmet_1': {'1': 617, '2': 2053, '3': 1345}, 'DARKO': {'1': 351, '2': 351, '3': 190, '4': 410, '5': 466, '6': 404, '7': 194}, 'Helmet_6': {'1': 2302, '2': 10912, '3': 7829, '4': 9674, '5': 8374}, 'Helmet_4': {'1': 2951, '2': 7223, '3': 8653, '4': 9500, '5': 9581}, 'Helmet_10': {'1': 3478, '2': 4874, '3': 4660, '4': 5057}, 'Helmet_7': {'1': 6017, '2': 9649, '3': 7314, '4': 12638, '5': 6760}, 'Helmet_5': {'1': 3419, '2': 1283, '3': 6006, '4': 7398, '5': 5809}, 'LO1': {'1': 2787, '2': 2634, '3': 2601, '4': 2444, '5': 3434, '6': 2121, '7': 3090, '8': 3820}}


In [11]:
def preprocess_df(raw_df: pd.DataFrame) -> pd.DataFrame:
    """interpolation and divide by 1000 to get measurements in meters"""
    preprocessed_df = raw_df.copy()
    trajectories_condition = preprocessed_df.columns[
            (preprocessed_df.columns.str.endswith(" X"))
            | ((preprocessed_df.columns.str.endswith(" Y")))
            | ((preprocessed_df.columns.str.endswith(" Z")))
        ]
    preprocessed_df[trajectories_condition] = preprocessed_df[trajectories_condition].interpolate()
    preprocessed_df[trajectories_condition] /= 1000
    return preprocessed_df

In [12]:
sync_df[sync_df.columns[sync_df.columns.str.endswith(" X")]].columns

Index(['DARKO - 1 X', 'DARKO - 2 X', 'DARKO - 3 X', 'DARKO - 4 X',
       'DARKO - 5 X', 'DARKO - 6 X', 'DARKO - 7 X', 'Helmet_1 - 1 X',
       'Helmet_1 - 2 X', 'Helmet_1 - 3 X', 'Helmet_10 - 1 X',
       'Helmet_10 - 2 X', 'Helmet_10 - 3 X', 'Helmet_10 - 4 X',
       'Helmet_2 - 1 X', 'Helmet_2 - 2 X', 'Helmet_2 - 3 X', 'Helmet_2 - 4 X',
       'Helmet_4 - 1 X', 'Helmet_4 - 2 X', 'Helmet_4 - 3 X', 'Helmet_4 - 4 X',
       'Helmet_4 - 5 X', 'Helmet_5 - 1 X', 'Helmet_5 - 2 X', 'Helmet_5 - 3 X',
       'Helmet_5 - 4 X', 'Helmet_5 - 5 X', 'Helmet_6 - 1 X', 'Helmet_6 - 2 X',
       'Helmet_6 - 3 X', 'Helmet_6 - 4 X', 'Helmet_6 - 5 X', 'Helmet_7 - 1 X',
       'Helmet_7 - 2 X', 'Helmet_7 - 3 X', 'Helmet_7 - 4 X', 'Helmet_7 - 5 X',
       'LO1 - 1 X', 'LO1 - 2 X', 'LO1 - 3 X', 'LO1 - 4 X', 'LO1 - 5 X',
       'LO1 - 6 X', 'LO1 - 7 X', 'LO1 - 8 X'],
      dtype='object')

In [13]:
preprocessed_trajectories = preprocess_df(sync_df)

In [14]:
preprocessed_trajectories[
    preprocessed_trajectories.columns[
        preprocessed_trajectories.columns.str.contains("G3D")
    ]
].isna().sum() / preprocessed_trajectories.shape[0]

Helmet_10 TB2_G3D_X    0.777242
Helmet_10 TB2_G3D_Y    0.777242
Helmet_10 TB2_G3D_Z    0.777242
Helmet_5 TB3_G3D_X     0.503856
Helmet_5 TB3_G3D_Y     0.503856
Helmet_5 TB3_G3D_Z     0.503856
dtype: float64

In [15]:
moving_agents = preprocessed_trajectories.columns[
    (preprocessed_trajectories.columns.str.startswith("Helmet"))
    | (preprocessed_trajectories.columns.str.startswith("LO1"))
].tolist()
moving_agents_labels = set(map(lambda x: x.split(" - ")[0], moving_agents))
moving_agents_labels = filter(lambda x: len(x.split(" ")) == 1, moving_agents_labels)

In [16]:
def extract_features(
    out_df: pd.DataFrame,
    magents_labels,
    darko_label,
) -> pd.DataFrame:
    """Extract features to be used in the profiles section such as speed

    Parameters
    ----------
    input_df
        raw pandas DataFrame
    magents_labels
        moving agents labels
    darko_label
        darko robot label

    Returns
    -------
        pandas DataFrame with the respective features computed
    """
    magents_labels = (
        [magents_labels] if isinstance(magents_labels, str) else magents_labels
    )
    elements_labels = magents_labels + [darko_label] if darko_label else magents_labels

    out_df = SpatioTemporalFeatures.get_speed(
        out_df,
        time_col_name="Time",
        element_name=elements_labels,
    )

    out_df = out_df.reset_index()
    return out_df

In [17]:
moving_agents_cols = preprocessed_trajectories.columns[(preprocessed_trajectories.columns.str.endswith(" X"))
            | ((preprocessed_trajectories.columns.str.endswith(" Y")))
            | ((preprocessed_trajectories.columns.str.endswith(" Z")))]

In [18]:
moving_agents_cols

Index(['DARKO - 1 X', 'DARKO - 1 Y', 'DARKO - 1 Z', 'DARKO - 2 X',
       'DARKO - 2 Y', 'DARKO - 2 Z', 'DARKO - 3 X', 'DARKO - 3 Y',
       'DARKO - 3 Z', 'DARKO - 4 X',
       ...
       'LO1 - 5 Z', 'LO1 - 6 X', 'LO1 - 6 Y', 'LO1 - 6 Z', 'LO1 - 7 X',
       'LO1 - 7 Y', 'LO1 - 7 Z', 'LO1 - 8 X', 'LO1 - 8 Y', 'LO1 - 8 Z'],
      dtype='object', length=138)

In [19]:
features_df = extract_features(
        preprocessed_trajectories[["Time"] + moving_agents_cols.tolist()].copy(),
        magents_labels=list(moving_agents_labels),
        darko_label="DARKO",
    )

2023-05-17 18:12:44.031 INFO    magni_dash.data_preprocessing.spatio_temporal_features: {'Helmet_7 - 5 X_delta', 'Helmet_7 - 5 Y_delta', 'LO1 - 3 X_delta', 'Helmet_5 - 3 Z_delta', 'Helmet_7 - 3 X_delta', 'Helmet_6 - 5 X_delta', 'Helmet_5 - 2 Z_delta', 'Helmet_4 - 4 Y_delta', 'Helmet_2 - 4 Y_delta', 'Helmet_5 - 4 Z_delta', 'DARKO - 6 X_delta', 'DARKO - 3 Y_delta', 'Helmet_2 - 1 Z_delta', 'Helmet_1 - 2 X_delta', 'Helmet_7 - 5 Z_delta', 'Helmet_1 - 1 X_delta', 'Helmet_6 - 2 Z_delta', 'LO1 - 5 Y_delta', 'Helmet_4 - 4 Z_delta', 'Helmet_10 - 1 Y_delta', 'Helmet_10 - 1 Z_delta', 'Helmet_7 - 1 Z_delta', 'LO1 - 6 Y_delta', 'DARKO - 7 Y_delta', 'Helmet_7 - 4 Z_delta', 'Helmet_10 - 3 Z_delta', 'Helmet_10 - 2 Y_delta', 'DARKO - 3 X_delta', 'Helmet_5 - 1 Z_delta', 'DARKO - 3 Z_delta', 'LO1 - 3 Y_delta', 'Helmet_6 - 1 Z_delta', 'Helmet_7 - 4 Y_delta', 'Helmet_10 - 1 X_delta', 'Helmet_5 - 5 Y_delta', 'LO1 - 1 Z_delta', 'DARKO - 1 Y_delta', 'Helmet_7 - 3 Z_delta', 'Helmet_4 - 3 Y_delta', 'Helmet_6 - 2

In [20]:
features_df.shape

(23860, 240)

In [21]:
features_cat = preprocessed_trajectories.join(features_df)

In [22]:
rotations_cols = features_cat[
    features_cat.columns[features_cat.columns.str.contains(r"R(\d)")]
].columns.tolist()
rotations_cols

/tmp/ipykernel_56774/2502953472.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  features_cat.columns[features_cat.columns.str.contains(r"R(\d)")]


['DARKO R0',
 'DARKO R1',
 'DARKO R2',
 'DARKO R3',
 'DARKO R4',
 'DARKO R5',
 'DARKO R6',
 'DARKO R7',
 'DARKO R8',
 'Helmet_1 R0',
 'Helmet_1 R1',
 'Helmet_1 R2',
 'Helmet_1 R3',
 'Helmet_1 R4',
 'Helmet_1 R5',
 'Helmet_1 R6',
 'Helmet_1 R7',
 'Helmet_1 R8',
 'Helmet_10 R0',
 'Helmet_10 R1',
 'Helmet_10 R2',
 'Helmet_10 R3',
 'Helmet_10 R4',
 'Helmet_10 R5',
 'Helmet_10 R6',
 'Helmet_10 R7',
 'Helmet_10 R8',
 'Helmet_2 R0',
 'Helmet_2 R1',
 'Helmet_2 R2',
 'Helmet_2 R3',
 'Helmet_2 R4',
 'Helmet_2 R5',
 'Helmet_2 R6',
 'Helmet_2 R7',
 'Helmet_2 R8',
 'Helmet_3 R0',
 'Helmet_3 R1',
 'Helmet_3 R2',
 'Helmet_3 R3',
 'Helmet_3 R4',
 'Helmet_3 R5',
 'Helmet_3 R6',
 'Helmet_3 R7',
 'Helmet_3 R8',
 'Helmet_4 R0',
 'Helmet_4 R1',
 'Helmet_4 R2',
 'Helmet_4 R3',
 'Helmet_4 R4',
 'Helmet_4 R5',
 'Helmet_4 R6',
 'Helmet_4 R7',
 'Helmet_4 R8',
 'Helmet_5 R0',
 'Helmet_5 R1',
 'Helmet_5 R2',
 'Helmet_5 R3',
 'Helmet_5 R4',
 'Helmet_5 R5',
 'Helmet_5 R6',
 'Helmet_5 R7',
 'Helmet_5 R8',
 'Helmet_6

In [23]:
features_filtered = features_cat[
    ["Frame"] + 
    features_cat.columns[
        (features_cat.columns.str.endswith("X"))
        | (features_cat.columns.str.endswith("Y"))
        | (features_cat.columns.str.endswith("Z"))
        | (features_cat.columns.str.endswith("speed (m/s)"))
    ].tolist()
    + rotations_cols
]

In [24]:
features_filtered

,Frame,DARKO - 1 X,DARKO - 1 Y,DARKO - 1 Z,DARKO - 2 X,DARKO - 2 Y,DARKO - 2 Z,DARKO - 3 X,DARKO - 3 Y,DARKO - 3 Z,...,Helmet_9 R8,LO1 R0,LO1 R1,LO1 R2,LO1 R3,LO1 R4,LO1 R5,LO1 R6,LO1 R7,LO1 R8
0,2,0.605676,1.332539,0.706138,0.558682,1.385699,0.327466,0.936693,0.562054,0.872280,...,NaN,-0.97705,-0.21289,-0.00744,0.21291,-0.97707,-0.00232,-0.00678,-0.00385,0.99997
1,3,0.605687,1.332543,0.706131,0.558644,1.385710,0.327417,0.936682,0.562035,0.872295,...,NaN,-0.97706,-0.21284,-0.00744,0.21287,-0.97708,-0.00230,-0.00678,-0.00383,0.99997
2,4,0.605665,1.332552,0.706107,0.558640,1.385708,0.327445,0.936886,0.562302,0.872263,...,NaN,-0.97717,-0.21234,-0.00758,0.21237,-0.97718,-0.00363,-0.00664,-0.00516,0.99996
3,5,0.605666,1.332562,0.706188,0.558642,1.385727,0.327422,0.936890,0.562322,0.872305,...,NaN,-0.97722,-0.21209,-0.00754,0.21212,-0.97724,-0.00352,-0.00662,-0.00504,0.99997
4,6,0.605715,1.332553,0.706072,0.558776,1.385782,0.327485,0.936896,0.562237,0.872256,...,NaN,-0.97665,-0.21470,-0.00759,0.21474,-0.97666,-0.00401,-0.00655,-0.00554,0.99996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23855,23857,-2.889115,-0.953349,0.696977,-2.835115,-0.906757,0.318487,-3.673524,-1.244002,0.863350,...,NaN,0.82410,0.56639,-0.00801,-0.56620,0.82408,0.01795,0.01677,-0.01026,0.99981
23856,23858,-2.886110,-0.954743,0.697099,-2.832301,-0.907913,0.318623,-3.670702,-1.245279,0.863376,...,NaN,0.82267,0.56848,-0.00712,-0.56831,0.82263,0.01748,0.01579,-0.01033,0.99982
23857,23859,-2.883548,-0.955789,0.697165,-2.829460,-0.909049,0.318672,-3.667847,-1.246681,0.863292,...,NaN,0.82070,0.57132,-0.00737,-0.57114,0.82067,0.01730,0.01593,-0.00999,0.99982
23858,23860,-2.880569,-0.957267,0.697274,-2.826575,-0.910380,0.318751,-3.668868,-1.248168,0.861944,...,NaN,0.81987,0.57250,-0.00687,-0.57236,0.81985,0.01584,0.01470,-0.00905,0.99985


# Visualization

In [25]:
import plotly.express as px
import plotly.graph_objects as go

In [26]:
features_filtered.columns[features_filtered.columns.str.contains("G3D")]

Index(['Helmet_10 TB2_G3D_X', 'Helmet_10 TB2_G3D_Y', 'Helmet_10 TB2_G3D_Z',
       'Helmet_5 TB3_G3D_X', 'Helmet_5 TB3_G3D_Y', 'Helmet_5 TB3_G3D_Z'],
      dtype='object')

In [27]:
HELMET_TO_VISUALIZE = "Helmet_10"

In [28]:
gaze_data = features_filtered[
    [
        f"{HELMET_TO_VISUALIZE} TB2_G3D_X",
        f"{HELMET_TO_VISUALIZE} TB2_G3D_Y",
        f"{HELMET_TO_VISUALIZE} TB2_G3D_Z",
    ]
].dropna()
gaze_data.shape

(5315, 3)

In [29]:
centroids_data = features_filtered[features_filtered.columns[features_filtered.columns.str.contains(f"{HELMET_TO_VISUALIZE} Centroid")]].dropna()
centroids_data.shape

(14147, 3)

In [30]:
centroids_data

,Helmet_10 Centroid_X,Helmet_10 Centroid_Y,Helmet_10 Centroid_Z
0,-2040.32922,-3194.35476,1880.25538
1,-2039.69035,-3198.14628,1880.25543
2,-2038.53058,-3201.69669,1880.35356
3,-2038.00564,-3205.22691,1880.53473
4,-2037.44637,-3208.50056,1879.75504
...,...,...,...
23855,-2225.49303,1354.72159,1864.04202
23856,-2238.54777,1350.35759,1861.69070
23857,-2251.54340,1345.99603,1859.54974
23858,-2264.51253,1341.62703,1857.52351


In [34]:
lo_info = GroupsInfo(
    element_id="LO1", markers_pattern_re=r"LO1 - (\d).*", label_sep=" - "
)
darko_info = GroupsInfo(
    element_id="DARKO", markers_pattern_re=r"DARKO - (\d).*", label_sep=" - "
)
helmets_info = GroupsInfo(
    element_id="Helmet", markers_pattern_re=r"Helmet_(\d+ - \d).*", label_sep="_"
)
tobii_info = GroupsInfo(
    element_id="Helmet", markers_pattern_re=r"Helmet_(\d+ TB\d)_G3D.*", label_sep="_"
)
centroids_info = GroupsInfo(
        element_id="Helmet",
        markers_pattern_re=r"Helmet_(\d+) Centroid_.*",
        label_sep="_",
    )
rotations_info = GroupsInfo(
        element_id="Helmet",
        markers_pattern_re=r"Helmet_(\d+) R.*",
        label_sep="_",
    )


In [35]:
def transform_df2plotly(
    input_df: pd.DataFrame, groups_info
) -> pd.DataFrame:
    """Transform a dataframe into the plotly best suited format
    |   Frame    |   X (m)  |   Y (m)  |   eid   |   mid   |

    being `eid` the element identifier (e.g. Helmet, DARKO, etc), and `mid` the marker identifier

    Parameters
    ----------
    input_df
        input pandas DataFrame
    element_id
        see eid explanation above
    markers_pattern_re
        regex to groupby element id and markers
    sep
        separation used in col name form element id and marker id

    Returns
    -------
        Transformed pandas DataFrame
    """
    groups_info = [groups_info] if isinstance(groups_info, GroupsInfo) else groups_info
    groups = []
    for group_info in groups_info:
        element_id = group_info.element_id
        elements_grouped = input_df.groupby(
            input_df.columns.str.extract(group_info.markers_pattern_re, expand=False),
            axis=1,
        )
        for group_name, group in elements_grouped:
            if element_id == "Helmet" and len(group_name.split("-")) == 1:
                tobii = len(group_name.split(" ")) == 2
                # eyt or centroids
                _mapping_cols = (
                    get_mapping_cols_tobii(
                        element_id,
                        group_name,
                        group_info.label_sep,
                    )
                    if tobii
                    else get_mapping_cols_centroids(
                        element_id, group_name, group_info.label_sep
                    )
                )
                group = group.rename(_mapping_cols, axis=1)
                eid = element_id + "_" + group_name.split(" ")[0]
            else:
                _mapping_cols = get_mapping_cols(
                    element_id, group_name, group_info.label_sep
                )
                group = group.rename(_mapping_cols, axis=1)
                eid = (
                    element_id + "_" + group_name.split(" - ")[0]
                    if element_id == "Helmet"
                    else element_id
                )
                mid = group_name.split(" - ")[1] if element_id == "Helmet" else group_name
                group["mid"] = mid
            group["eid"] = eid
            groups.append(group)
    out_df = pd.concat(groups, axis=0)
    return out_df

In [36]:
transformed = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[tobii_info],
    )

In [37]:
transformed

,TB_G3D X (m),TB_G3D Y (m),TB_G3D Z (m),eid
0,NaN,NaN,NaN,Helmet_10
1,-136.2645,-135.0888,346.0942,Helmet_10
2,NaN,NaN,NaN,Helmet_10
3,-132.5629,-136.0257,347.4366,Helmet_10
4,NaN,NaN,NaN,Helmet_10
...,...,...,...,...
23855,NaN,NaN,NaN,Helmet_5
23856,-19.3590,-54.7595,709.2499,Helmet_5
23857,NaN,NaN,NaN,Helmet_5
23858,-18.6287,-98.5383,1250.0811,Helmet_5


In [39]:
transformed_cent = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[centroids_info],
    )

In [48]:
transformed_cent = transformed_cent.sort_index()

In [49]:
transformed_cent

,Centroid X (m),Centroid Y (m),Centroid Z (m),eid
0,NaN,NaN,NaN,Helmet_1
0,301.18822,2969.91989,1786.75414,Helmet_2
0,NaN,NaN,NaN,Helmet_4
0,NaN,NaN,NaN,Helmet_5
0,NaN,NaN,NaN,Helmet_6
...,...,...,...,...
23926,NaN,NaN,NaN,Helmet_6
23926,NaN,NaN,NaN,Helmet_1
23926,NaN,NaN,NaN,Helmet_4
23926,8927.61106,-614.97743,1610.19733,Helmet_2


In [50]:
transformed_rotations = transform_df2plotly(
        input_df=features_filtered.copy(),
        groups_info=[rotations_info],
    )

In [51]:
transformed_rotations = transformed_rotations.sort_index()

In [52]:
transformed_rotations

,R0,R1,R2,R3,R4,R5,R6,R7,R8,eid
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_1
0,-0.85268,-0.51676,-0.07674,0.52227,-0.83966,-0.14902,0.01257,-0.16714,0.98585,Helmet_2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_6
...,...,...,...,...,...,...,...,...,...,...
23926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_6
23926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_1
23926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Helmet_4
23926,0.66726,0.61238,-0.42398,-0.65820,0.75123,0.04918,0.34863,0.24625,0.90433,Helmet_2


In [62]:
(transformed_rotations.eid.values == transformed_cent.eid.values).sum() == len(transformed_rotations)

True

In [76]:
cent_rotations = pd.concat([transformed_cent, transformed_rotations.drop("eid", axis = 1)], axis = 1)

In [77]:
cent_rotations

,Centroid X (m),Centroid Y (m),Centroid Z (m),eid,R0,R1,R2,R3,R4,R5,R6,R7,R8
0,NaN,NaN,NaN,Helmet_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,301.18822,2969.91989,1786.75414,Helmet_2,-0.85268,-0.51676,-0.07674,0.52227,-0.83966,-0.14902,0.01257,-0.16714,0.98585
0,NaN,NaN,NaN,Helmet_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,Helmet_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23926,NaN,NaN,NaN,Helmet_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23926,NaN,NaN,NaN,Helmet_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23926,NaN,NaN,NaN,Helmet_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23926,8927.61106,-614.97743,1610.19733,Helmet_2,0.66726,0.61238,-0.42398,-0.65820,0.75123,0.04918,0.34863,0.24625,0.90433


In [78]:
transformed_trajectories = transform_df2plotly(
        input_df=features_filtered[features_filtered.columns[features_filtered.columns.str.contains("")]].copy(),
        groups_info=[helmets_info],
    )
transformed_trajectories = transformed_trajectories.sort_index()

In [79]:
transformed_trajectories

,X (m),Y (m),Z (m),speed (m/s),mid,eid
0,NaN,NaN,NaN,NaN,1,Helmet_1
0,NaN,NaN,NaN,NaN,2,Helmet_10
0,NaN,NaN,NaN,NaN,2,Helmet_6
0,0.359082,3.059350,1.799919,NaN,3,Helmet_2
0,NaN,NaN,NaN,NaN,4,Helmet_10
...,...,...,...,...,...,...
23926,-8.317673,1.921456,1.497061,0.000000,5,Helmet_5
23926,8.163636,0.869036,1.961281,1.519131,1,Helmet_10
23926,8.933929,-0.531036,1.544316,0.787617,1,Helmet_2
23926,8.504732,-0.501440,1.683069,0.000000,5,Helmet_6


In [80]:
transformed.sort_index()

,TB_G3D X (m),TB_G3D Y (m),TB_G3D Z (m),eid
0,NaN,NaN,NaN,Helmet_10
0,NaN,NaN,NaN,Helmet_5
1,NaN,NaN,NaN,Helmet_5
1,NaN,NaN,NaN,Helmet_10
2,NaN,NaN,NaN,Helmet_10
...,...,...,...,...
23924,NaN,NaN,NaN,Helmet_10
23925,NaN,NaN,NaN,Helmet_5
23925,NaN,NaN,NaN,Helmet_10
23926,NaN,NaN,NaN,Helmet_10


In [68]:
def filter_best_markers(elements_cat_df: pd.DataFrame, nan_counter_by_marker):
    elements_filtered_by_best_marker = []
    for instance_id, nans_counter in nan_counter_by_marker.items():
        best_marker_id = min(
            nans_counter,
            key=nans_counter.get,
        )
        elements_filtered_by_best_marker.append(
            elements_cat_df[
                (elements_cat_df.eid == instance_id)
                & (elements_cat_df.mid == best_marker_id)
            ]
        )
    out_df = pd.concat(elements_filtered_by_best_marker, axis=0)
    out_df = out_df.sort_index().reset_index()
    return out_df

In [69]:
best_makers_df = filter_best_markers(
        elements_cat_df=transformed_trajectories.copy(), nan_counter_by_marker=best_markers
    )

In [70]:
best_makers_df

,index,X (m),Y (m),Z (m),speed (m/s),eid,mid
0,0,NaN,NaN,NaN,NaN,Helmet_4,1
1,0,-8.414543,1.655896,1.353073,NaN,Helmet_5,2
2,0,0.236878,2.938533,1.829624,NaN,Helmet_2,2
3,0,NaN,NaN,NaN,NaN,Helmet_7,1
4,0,-7.037703,-2.437462,1.812973,NaN,Helmet_1,2
...,...,...,...,...,...,...,...
167484,23926,8.561741,0.561951,1.885170,0.603637,Helmet_1,2
167485,23926,7.683523,0.036912,1.863660,0.876024,Helmet_4,1
167486,23926,8.998510,-0.572453,1.623232,1.073293,Helmet_2,2
167487,23926,-8.583218,2.080728,1.261840,0.960324,Helmet_5,2


In [71]:
ORIGIN = (0,0,0)
X = (1, 0, 0)
Y = (0, 1, 0)
Z = (0, 0, 1)
COLOR1 = 'red'
COLOR2 = 'green'
COLOR3 = 'blue'
COLOR4 = 'orange'

<TB2_G3D_X, TB2_G3D_Y, TB2_G3D_Z> is the transformation from the helmet frame to a convergence point
of the eye gaze.

In [73]:
# fig = px.scatter_3d(
#     transformed[:40000],
#     x="TB_G3D X (m)",
#     y="TB_G3D Y (m)",
#     z="TB_G3D Z (m)",
#     color="eid"
# )
fig = go.Figure([])
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], X[0], None],
        y=[ORIGIN[1], X[1], None],
        z=[ORIGIN[2], X[2], None],
        mode="lines",
        line=dict(color="red", width=5),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], Y[0], None],
        y=[ORIGIN[1], Y[1], None],
        z=[ORIGIN[2], Y[2], None],
        mode="lines",
        line=dict(color="green", width=5),
        showlegend=False,
    )
)
fig.add_trace(
    go.Scatter3d(
        x=[ORIGIN[0], Z[0], None],
        y=[ORIGIN[1], Z[1], None],
        z=[ORIGIN[2], Z[2], None],
        mode="lines",
        line=dict(color="blue", width=5),
        showlegend=False,
    )
)

fig.add_cone(
    x=[1],
    y=[0],
    z=[0],
    u=[1],
    v=[0],
    w=[0],
    colorscale=[[0, COLOR1], [1, COLOR1]],
    showscale=False,
)

fig.add_cone(
    x=[0],
    y=[1],
    z=[0],
    u=[0],
    v=[1],
    w=[0],
    colorscale=[[0, COLOR2], [1, COLOR2]],
    showscale=False,
)

fig.add_cone(
    x=[0],
    y=[0],
    z=[Z[2]],
    u=[0],
    v=[0],
    w=[Z[2]],
    colorscale=[[0, COLOR3], [1, COLOR3]],
    showscale=False,
)